In [ ]:
from bs4 import BeautifulSoup
import requests 
import re
from time import sleep
import pandas as pd
import time
from tqdm import tqdm
#引数はページの値（0,1)1ページ目から2ページ目の高配当株ランキングの3.75%以上のものを取得してその企業コードを格納
securities_codes = get_securities_codes(30,33)


#securities_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
closing_urls = []
# for code in securities_codes[:3]:
for code in securities_codes:
    closing_urls.append(get_closing_url(code))
#-------------------------------------------------------------------------一先ず終了2023/01/05----------------------------------------------------------------------#
#上記を満たした企業コードの決算ページxxxxxx/resultsにアクセスし企業名と条件の財務の値を取得
focus_url=[]
for closing_url in closing_urls:
    url='https://irbank.net{}'.format(closing_url)
    sleep(2)
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
text=[]
for element in tagget:
    for a in element.find_all('a'):
        a.decompose()
    text.append(element)
   #-------------------------------------------------------------------------ひとまず終了2023/01/05-------------------------------------------------------------------------#
target_h2 = []#正規は８項目
for t in text:
    if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t:
        target_h2.append(t)
test_lists=[]
if len(target_h2)==8:
    for i in target_h2:
        bro=i.find_next_sibling()
        ss = bro.find_all('span', class_='text')
        test_lists.append(ss)

if test_lists==8:
#-------------------------------------------------------------------------ひとまず終了2023/01/05-------------------------------------------------------------------------#
    d_list=[]
#--------------------------------------------------------------------- 1:ここから売上高2023/01/05-------------------------------------------------------------------------#
#     if isinstance(test_lists[0], list):
    sales_list=[sale.text for sale in test_lists[0]]


    #zfill完了
    eval_sale_list = []
    for sale in sales_list:
        eval_sale_list.append(convert_sales_figure(sale))
    #採点完了
    uriagedaka_stock = check_uriagedaka(eval_sale_list)

#-------------------------------------------------------------------------2:ここからEPS2023/01/05--------------------------------------------------------------------------#
    eps_list=[ep.text for ep in test_lists[1]]
    #zfill完了
    zeps_list = []
    for ep in eps_list:
        zeps_list.append(convert_eps(ep))
    eps_stock = check_eps(zeps_list)

#-------------------------------------------------------------------------3:ここから営業利益率2023/01/05- -------------------------------------------------------------------------#     

    op_list=[op.text for op in test_lists[2]]
    zop_list=[]
    for s in op_list:
        hennkan = s.split('%')[0]  # '15.65' のような文字列を取得
        zop_list.append(eval(hennkan))

    eiri_stock = check_eiri(zop_list)


#-------------------------------------------------------------------------4:ここから自己資本利益率2023/01/05-------------------------------------------------------------------------#

    operation_list=[operation.text for operation in test_lists[3]]
    zoption_list=[]
    for s in operation_list:
        result = floatparcent_hennkan(s)  # hennkan 関数を呼び出す
        zoption_list.append(result)

        zikosihon_stock = check_zikosihon(zoption_list)


               #zoption_list 変数に格納された要素を、個別に確認する
    #     for i, value in enumerate(zoption_list):
    #         # value 変数が 40.00 以上であるかどうかを判定する
    #         if float(40.00) <= float(value):
    #             print(f'index {i}: OK')
    #         else:
    #             print(f'index {i}: NOT')




#-------------------------------------------------------------------------5:ここから現金2023/01/05-------------------------------------------------------------------------#
    cash_list=[cash.text for cash in test_lists[5]]
      #zfill完了
    eval_cash_list = []
    for cash in cash_list:
        eval_cash_list.append(convert_sales_figure(cash))
        genkin_stock = check_genkin(eval_cash_list)
       #-------------------------------------------------------------------------6:ここからCF -------------------------------------------------------------------------#

    cf_flow_list=[cfflow.text for cfflow in test_lists[4]]
    #zfill完了
    eval_cf_flow_list = []
    for flow in cf_flow_list:
        eval_cf_flow_list.append(convert_flow_figure(flow))
    # モジュール内の関数を呼び出す
        cflow_stock = check_cflow_order(eval_cf_flow_list)

    #-------------------------------------------------------------------------7:ここから一株当たり-------------------------------------------------------------------------#
        #zfill完了

    stock_list=[stock.text for stock in test_lists[6]]

    zstock_list=[]
    for s in stock_list:
        result = convert_stock(s)  # hennkan 関数を呼び出す
        zstock_list.append(result)                          
    hitokabu_stock=check_zstock_order(zstock_list)

        #-------------------------------------------------------------------------8:配当性向-------------------------------------------------------------------------#
    return_list=[return_money.text for return_money in test_lists[7]]
    #zfill完了
    zreturn_list = []
    for return_money in return_list:
        zreturn_list.append(convert_return(return_money))

    seikou_stock=check_zreturn_range(zreturn_list)
        #-------------------------------------------------------------------------ひとまず終了------------------------------------------------------------------------#
    analyze=[]
    analyze.append(uriagedaka_stock)
    analyze.append(eps_stock)
    analyze.append(eiri_stock)
    analyze.append(zikosihon_stock)
    analyze.append(genkin_stock)
    analyze.append(cflow_stock)
    analyze.append(hitokabu_stock)
    analyze.append(seikou_stock)


    fine=[]
    for analy in analyze:
        if 'OK'in analy or len(analy)==1:
            fine.append('○')
    #     if len(analy)==1:
    #         fine.append('○')
        else:
            fine.append('X')
    if fine.count('○') >= 2:
        d_list.append(title)
        d_list.append(fine)
        df=pd.DataFrame(d_list)
        #to_csv()を使って、データフレームをCSV出力する
        df.to_csv('stock_target.csv',index=True,encoding='utf-8-sig',mode='a')
else:
    pass
from bs4 import BeautifulSoup
import requests 
import re
from time import sleep
import pandas as pd
import time
from tqdm import tqdm
#引数はページの値（0,1)1ページ目から2ページ目の高配当株ランキングの3.75%以上のものを取得してその企業コードを格納
securities_codes = get_securities_codes(30,33)


#securities_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
closing_urls = []
# for code in securities_codes[:3]:
for code in securities_codes:
    closing_urls.append(get_closing_url(code))
#-------------------------------------------------------------------------一先ず終了2023/01/05----------------------------------------------------------------------#
#上記を満たした企業コードの決算ページxxxxxx/resultsにアクセスし企業名と条件の財務の値を取得
focus_url=[]
for closing_url in closing_urls:
    url='https://irbank.net{}'.format(closing_url)
    sleep(2)
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
text=[]
for element in tagget:
    for a in element.find_all('a'):
        a.decompose()
    text.append(element)
   #-------------------------------------------------------------------------ひとまず終了2023/01/05-------------------------------------------------------------------------#
target_h2 = []#正規は８項目
for t in text:
    if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t:
        target_h2.append(t)
test_lists=[]
if len(target_h2)==8:
    for i in target_h2:
        bro=i.find_next_sibling()
        ss = bro.find_all('span', class_='text')
        test_lists.append(ss)

if test_lists==8:
#-------------------------------------------------------------------------ひとまず終了2023/01/05-------------------------------------------------------------------------#
    d_list=[]
#--------------------------------------------------------------------- 1:ここから売上高2023/01/05-------------------------------------------------------------------------#
#     if isinstance(test_lists[0], list):
    sales_list=[sale.text for sale in test_lists[0]]


    #zfill完了
    eval_sale_list = []
    for sale in sales_list:
        eval_sale_list.append(convert_sales_figure(sale))
    #採点完了
    uriagedaka_stock = check_uriagedaka(eval_sale_list)

#-------------------------------------------------------------------------2:ここからEPS2023/01/05--------------------------------------------------------------------------#
    eps_list=[ep.text for ep in test_lists[1]]
    #zfill完了
    zeps_list = []
    for ep in eps_list:
        zeps_list.append(convert_eps(ep))
    eps_stock = check_eps(zeps_list)

#-------------------------------------------------------------------------3:ここから営業利益率2023/01/05- -------------------------------------------------------------------------#     

    op_list=[op.text for op in test_lists[2]]
    zop_list=[]
    for s in op_list:
        hennkan = s.split('%')[0]  # '15.65' のような文字列を取得
        zop_list.append(eval(hennkan))

    eiri_stock = check_eiri(zop_list)


#-------------------------------------------------------------------------4:ここから自己資本利益率2023/01/05-------------------------------------------------------------------------#

    operation_list=[operation.text for operation in test_lists[3]]
    zoption_list=[]
    for s in operation_list:
        result = floatparcent_hennkan(s)  # hennkan 関数を呼び出す
        zoption_list.append(result)

        zikosihon_stock = check_zikosihon(zoption_list)


               #zoption_list 変数に格納された要素を、個別に確認する
    #     for i, value in enumerate(zoption_list):
    #         # value 変数が 40.00 以上であるかどうかを判定する
    #         if float(40.00) <= float(value):
    #             print(f'index {i}: OK')
    #         else:
    #             print(f'index {i}: NOT')




#-------------------------------------------------------------------------5:ここから現金2023/01/05-------------------------------------------------------------------------#
    cash_list=[cash.text for cash in test_lists[5]]
      #zfill完了
    eval_cash_list = []
    for cash in cash_list:
        eval_cash_list.append(convert_sales_figure(cash))
        genkin_stock = check_genkin(eval_cash_list)
       #-------------------------------------------------------------------------6:ここからCF -------------------------------------------------------------------------#

    cf_flow_list=[cfflow.text for cfflow in test_lists[4]]
    #zfill完了
    eval_cf_flow_list = []
    for flow in cf_flow_list:
        eval_cf_flow_list.append(convert_flow_figure(flow))
    # モジュール内の関数を呼び出す
        cflow_stock = check_cflow_order(eval_cf_flow_list)

    #-------------------------------------------------------------------------7:ここから一株当たり-------------------------------------------------------------------------#
        #zfill完了

    stock_list=[stock.text for stock in test_lists[6]]

    zstock_list=[]
    for s in stock_list:
        result = convert_stock(s)  # hennkan 関数を呼び出す
        zstock_list.append(result)                          
    hitokabu_stock=check_zstock_order(zstock_list)

        #-------------------------------------------------------------------------8:配当性向-------------------------------------------------------------------------#
    return_list=[return_money.text for return_money in test_lists[7]]
    #zfill完了
    zreturn_list = []
    for return_money in return_list:
        zreturn_list.append(convert_return(return_money))

    seikou_stock=check_zreturn_range(zreturn_list)
        #-------------------------------------------------------------------------ひとまず終了------------------------------------------------------------------------#
    analyze=[]
    analyze.append(uriagedaka_stock)
    analyze.append(eps_stock)
    analyze.append(eiri_stock)
    analyze.append(zikosihon_stock)
    analyze.append(genkin_stock)
    analyze.append(cflow_stock)
    analyze.append(hitokabu_stock)
    analyze.append(seikou_stock)


    fine=[]
    for analy in analyze:
        if 'OK'in analy or len(analy)==1:
            fine.append('○')
    #     if len(analy)==1:
    #         fine.append('○')
        else:
            fine.append('X')
    if fine.count('○') >= 2:
        d_list.append(title)
        d_list.append(fine)
        df=pd.DataFrame(d_list)
        #to_csv()を使って、データフレームをCSV出力する
        df.to_csv('stock_target.csv',index=True,encoding='utf-8-sig',mode='a')
else:
    pass


In [744]:
# uriagedaka_stock,eps_stock,eiri_stock,zikosihon_stock,genkin_stock,cflow_stock,hitokabu_stock,seikou_stock
d_list

['3834 朝日ネット']

In [47]:
df=pd.DataFrame(d_list)
df.head()

NameError: name 'd_list' is not defined

In [728]:
def get_securities_codes(start_page, end_page):
    url='https://minkabu.jp/financial_item_ranking/dividend_yield?page={}'
    dd_list=[]
    for i in range(start_page, end_page+1):
        target_url=url.format(i)
        r=requests.get(target_url)
        sleep(5)
        soup=BeautifulSoup(r.text,'html.parser')
        tab=soup.find('table',class_='md_table rnk_li')
        conts=tab.find_all('tr')
        del conts[0]

        for cont in conts:
            rank=cont.find('span',class_='ranking_no fsize_l').text
            code=cont.find('div',class_='md_sub').text
            name=cont.find('div',class_='fwb w90p').text
            y=cont.find('td',class_='tar cur vamd').text
            yeild= y.strip().replace('%', '')
            dic={
                'ランキング':rank,
                '証券コード':code,
                '企業名':name,
                '配当利回り':yeild,
            }
            if float(yeild) >= 3.75:
                dd_list.append(dic)
    securities_codes = [d['証券コード'] for d in dd_list]
    return securities_codes





def float_hennkan(s):
    # *だけの時
    if '*' in s:
        s = s.split('円')[0]  # '15.65' のような文字列を取得
        s = s.replace('*', '')  # * を削除
        if '.'in s:
            integer_part, decimal_part =s.split('.')  # '15' と '65' のような文字列を取得
            integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
            decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
            hennkan_result= f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result
        
        
    #小数点の時
    elif '.' in s :
        number_str = s.split('円')[0]  # '15.65' のような文字列を取得
        integer_part, decimal_part = number_str.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result = f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result
      

    else:
        # 整数部と小数部が分離されていない場合
        number_str = s.split('円')[0]  # '86' のような文字列を取得
        integer_part = number_str.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part}.00"
        return hennkan_result
    

#     if '.' in s:
#         number_str1 = kn .split('円')[0]  # '86' のような文字列を取得
#         integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
#         hennkan_result = f"{integer_part1}.00"
#         return hennkan_result
#     *だけの時
#     elif '*' in s:
#         kn = s.split('*')[1]
#         number_str1 = kn .split('円')[0]  # '86' のような文字列を取得
#         integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
#         hennkan_result = f"{integer_part1}.00"
#         return hennkan_result



def convert_sales_figure(sales_figure: str) -> int:
    #兆しかない時
    if '兆' in sales_figure and '億' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('兆', '*10**12')
        return eval(hennkan)
    #億しかない時
    elif '億' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('億', '*10**8')
        return eval(hennkan)
    #万しかない時
    elif '万' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('万', '*10**4')
        return eval(hennkan)
    #兆と億の時
    elif'兆'in sales_figure and '億' in sales_figure:
        hennkan = sales_figure.replace('兆', '*10**12+').replace('億','*10**8')
        return eval(hennkan)
    #億と万の時
    elif'億'in sales_figure and '万' in sales_figure:
        hennkan = sales_figure.replace('億', '*10**8+').replace('万','*10**4')
        return eval(hennkan)
    else:
        hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sales_figure)
        return eval(hennkan)



def convert_eps(eps_figure: str) -> int:
    number_str = eps_figure.split('円')[0]
    hennkan = number_str + '*100'
    return eval(hennkan)



def convert_stock(stock_price: str) -> str:
    if '.' in stock_price:
        number_str = stock_price.split('円')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    elif '*' in stock_price:
        kn = stock_price.split('*')[1]
        number_str1 = kn .split('円')[0]
        integer_part1 = number_str1.zfill(2)
        return f"{integer_part1}.00"
    else:
        number_str = stock_price.split('円')[0]
        integer_part = number_str.zfill(2)
        return f"{integer_part}.00"



def convert_return(return_figure: str) -> str:
    if '.' in return_figure:
        number_str = return_figure.split('%')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"

    else:
        integer_part = return_figure.split('%')[0]
        integer_part = integer_part.zfill(2)
        return f"{integer_part}.00"

def process_cashes(cashes):
    cash_list = [cash.text for cash in cashes]
    eval_cash_list = []
    for cash in cash_list:
        # 兆しかない時
        if '兆' in cash and '億' not in cash and '万' not in cash:
            hennkan = cash.replace('兆', '*10**12')
            eval_cash_list.append(eval(hennkan))
        # 億しかない時
        elif '億' in cash and '兆' not in cash and '万' not in cash:
            hennkan = cash.replace('億', '*10**8')
            eval_cash_list.append(eval(hennkan))
        # 万しかない時
        elif '万' in cash and '兆' not in cash and '万' not in cash:
            hennkan = cash.replace('万', '*10**4')
            eval_cash_list.append(eval(hennkan))
        # 兆と億の時
        elif '兆' in cash and '億' in cash:
            hennkan = cash.replace('兆', '*10**12+').replace('億','*10**8')
            eval_cash_list.append(eval(hennkan))
        # 億と万の時
        elif '億' in cash and '万' in cash:
            hennkan = cash.replace('億', '*10**8+').replace('万','*10**4')
            eval_cash_list.append(eval(hennkan))
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sale)
            eval_cash_list.append(eval(hennkan))
    return eval_cash_list


import re

def process_cfflows(cfflows):
    cf_flow_list = [cfflow.text for cfflow in cfflows]
    eval_cf_flow_list = []
    for flow in cf_flow_list:
        # 兆しかない時
        if '兆' in flow and '億' not in flow and '万' not in flow:
            hennkan = flow.replace('兆', '*10**12')
            eval_cf_flow_list.append(eval(hennkan))
        # 億しかない時
        elif '億' in flow and '兆' not in flow and '万' not in flow:
            hennkan = flow.replace('億', '*10**8')
            eval_cf_flow_list.append(eval(hennkan))
        # 万しかない時
        elif '万' in flow and '兆' not in flow and '万' not in flow:
            hennkan = flow.replace('万', '*10**4')
            eval_cf_flow_list.append(eval(hennkan))
        # 兆と億の時
        elif '兆' in flow and '億' in flow:
            hennkan = flow.replace('兆', '*10**12+').replace('億','*10**8')
            eval_cf_flow_list.append(eval(hennkan))
        # 億と万の時
        elif '億' in flow and '万' in flow:
            hennkan = flow.replace('億', '*10**8+').replace('万','*10**4')
            eval_cf_flow_list.append(eval(hennkan))
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sale)
            eval_cf_flow_list.append(eval(hennkan))
    return eval_cf_flow_list


def convert_japanese_financial_string(string):
    hennkan = string.replace('兆', '*10**12').replace('億', '*10**8').replace('万', '*10**4')
    return eval(hennkan)



def convert_flow_figure(flow_figure: str) -> int:
        #兆しかない時
        if '兆' in flow_figure and '億' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('兆', '*10**12')
            return eval(hennkan)         
        #億しかない時
        elif '億' in flow_figure and '兆' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('億', '*10**8')
            return eval(hennkan)             
        #万しかない時
        elif '万' in flow_figure and '兆' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('万', '*10**4')
            return eval(hennkan)
        #兆と億の時
        elif'兆'in flow_figure and '億' in flow_figure:
            hennkan = flow_figure.replace('兆', '*10**12+').replace('億','*10**8')
            return eval(hennkan)
        #億と万の時
        elif'億'in flow_figure and '万' in flow_figure:
            hennkan = flow_figure.replace('億', '*10**8+').replace('万','*10**4')
            return eval(hennkan)
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", flow_figure)
            return eval(hennkan)


def floatparcent_hennkan(s):
    if '.' in s:
        number_str = s.split('%')[0]  # '15.65' のような文字列を取得
        integer_part, decimal_part = number_str.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result = f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result

    else:
        # 整数部と小数部が分離されていない場合
        integer_part = s.split('%')[0]  # '86' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part}.00"
        return hennkan_result
def check_uriagedaka(eval_sale_list):
    result = all(int(eval_sale_list[i]) < int(eval_sale_list[i+1]) for i in range(len(eval_sale_list)-1))
    uriagedaka_stock = []
    if result:
        uriagedaka_stock.append('OK')
    else:
        for i in range(len(eval_sale_list)-1):
            if int(eval_sale_list[i]) >= int(eval_sale_list[i+1]):
                uriagedaka_stock.append(f'NOT (at index {i+1})')
    return uriagedaka_stock


def check_eps(zeps_list):
    result = all(int(zeps_list[i]) < int(zeps_list[i+1]) for i in range(len(zeps_list)-1))
    eps_stock = []
    if result:
        eps_stock.append('OK')
    else:
        for i in range(len(zeps_list)-1):
            if int(zeps_list[i]) >= int(zeps_list[i+1]):
                eps_stock.append(f'NOT (at index {i+1})')
    return eps_stock

def check_eiri(zop_list):
    zop_list = [float(x) for x in zop_list]
    resault = all(10 <= float(zop_list[i]) for i in range(len(zop_list)))
    eiri_stock = []
    if resault:
        eiri_stock.append('OK')
    else:
        for i in range(len(zop_list)):
            if 10 >= float(zop_list[i]):
                eiri_stock.append(f'NOT (at index {i})')
    return eiri_stock

def check_zikosihon(zoption_list):
    oolist = [float(x) for x in zoption_list]
    result = all(40 <= float(oolist[i]) for i in range(len(oolist)))
    zikosihon_stock = []
    if result:
        zikosihon_stock.append('OK')
    else:
        for i in range(len(oolist)):
            if 40 >= float(oolist[i]):
                zikosihon_stock.append(f'NOT (at index {i})')
    return zikosihon_stock


def check_genkin(eval_cash_list):
    result = all(int(eval_cash_list[i]) < int(eval_cash_list[i+1]) for i in range(len(eval_cash_list)-1))
    genkin_stock = []
    if result:
        genkin_stock.append('OK')
    else:
        for i in range(len(eval_cash_list)-1):
            if int(eval_cash_list[i]) >= int(eval_cash_list[i+1]):
                genkin_stock.append(f'NOT (at index {i+1})')
    return genkin_stock



def check_cflow_order(eval_cf_flow_list):
    result = all(int(eval_cf_flow_list[i]) < int(eval_cf_flow_list[i+1]) for i in range(len(eval_cf_flow_list)-1))
    cflow_stock=[]
    if result:
        cflow_stock.append('OK')
    else:
        for i in range(len(eval_cf_flow_list)-1):
            if int(eval_cf_flow_list[i]) >= int(eval_cf_flow_list[i+1]):
                cflow_stock.append(f'NOT (at index {i+1})')
    return cflow_stock

def check_zstock_order(zstock_list):
    result = all(float(zstock_list[i]) < float(zstock_list[i+1]) for i in range(len(zstock_list)-1))
    hitokabu_stock=[]
    if result:
        hitokabu_stock.append('OK')
    else:
        for i in range(len(zstock_list)-1):
            if float(zstock_list[i]) >= float(zstock_list[i+1]):
                hitokabu_stock.append(f'NOT (at index {i+1})')
    return hitokabu_stock

def check_zreturn_range(zreturn_list):
    int_zreturn_list = [float(x) for x in zreturn_list]
    result = all(30<int(int_zreturn_list[i]<50) for i in range(len(int_zreturn_list)))
    seikou_stock=[]
    if result:
        seikou_stock.append('OK')
    else:
        for i in range(len(int_zreturn_list)):
            if int(30>int_zreturn_list[i] or int_zreturn_list[i]>70):
                seikou_stock.append(f'NOT (at index {i})')
    return seikou_stock


def get_closing_url(code):
    url = f'https://irbank.net/{code}/ir'
    r = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(r.text,'html.parser')
    element = soup.select_one('div.csb.cc2 > ul:nth-of-type(2) > li:first-of-type')
    taget = element.find('a')
    link = taget.get('href')
    return link

def convert_sales(*args):
    eval_list = []
    for arg in args:
        eval_arg = []
        for value in arg:
            # 兆しかない時
            if '兆' in value and '億' not in value and '万' not in value:
                hennkan = value.replace('兆', '*10**12')
                eval_arg.append(eval(hennkan))
        eval_list.append(eval_arg)
    return eval_list


def convert_eps(eps_figure: str) -> int:
    number_str = eps_figure.split('円')[0]
    hennkan = number_str + '*100'
    return eval(hennkan)

def process_code(code):
    url = f'https://irbank.net/{code}/ir'
#     print(url)
    r=requests.get(url)
    sleep(2)
    soup=BeautifulSoup(r.text,'html.parser')
    # CSS セレクターを使用して要素を取得
    element = soup.select_one('div.csb.cc2 > ul:nth-of-type(2) > li:first-of-type')
    taget = element.find('a')
    link = taget.get('href')
    return link

In [ ]:

from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import requests 
import re
from selenium import webdriver
from time import sleep
import pandas as pd
# 処理を開始する時間を計測
# start = time.perf_counter()
closing_url=[]
for code in tqdm(securities_codes):
    url = f'https://irbank.net/{code}/ir'
#     print(url)
    r=requests.get(url)
    sleep(2)
    soup=BeautifulSoup(r.text,'html.parser')
    # CSS セレクターを使用して要素を取得
    element = soup.select_one('div.csb.cc2 > ul:nth-of-type(2) > li:first-of-type')
    taget = element.find('a')
    link = taget.get('href')
    closing_url.append(link)
#     print(link)
# # 処理を終了する時間を計測
# end = time.perf_counter()
# # 処理にかかった時間を算出
# elapsed_time = end - start
# print(f'elapsed time: {elapsed_time:.6f} [秒]')





